In [150]:
# Step 2

def identity_preprocess(str):
  return str

def readfile(path,preprocess=identity_preprocess):
  processed_txt=[]
  f = open(path, "r")
  for line in f:
    processed_txt= processed_txt + preprocess(line)
  return processed_txt 

def tokenize(s):
    s = s.strip().lower()
    s = ''.join(c if c.isalpha() else ' ' for c in s)
    # We replace all non-alpha characters with ' '
    s = s.replace('\n',' ')
    s = s.split()
    return s
  
# TODO: experiment with ntk tokenizers 




In [151]:
# Step 3

text = readfile('./data/36-0.txt',tokenize)
print(len(text))
word_corpus = list(set(text))
print(len(word_corpus))
# Convert list of words to list of chars, get unique chars with set
alphabet = list(set([c for word in word_corpus for c in word]))
print(len(alphabet))



64183
7072
28


In [152]:
# Step 4

def create_syms(alphabet):
    f = open("./chars.syms","w+")
    f.write(f'<epsilon>     0\n')  #for <epsilon> index 0
    for i in range(len(alphabet)):
        f.write(f'{alphabet[i]}     {i+50}\n')  #for other characters index i+50
        
create_syms(alphabet)
    

In [153]:
#Step 5
# We must create FST using shell

# arc format: src dest ilabel olabel [weight]
# final state format: state [weight]
# lines may occur in any order except initial state must be first line
# unspecified weights default to 0.0 (for the library-default Weight type) 

f = open('lev.fst',"w+")

def format_arc(src,dst,src_sym,dst_sym,w,f,acceptor=False):
    if not acceptor:
        f.write("{} {} {} {} {} \n".format(src,dst,src_sym,dst_sym,w))
    else:
        f.write("{} {} {} \n".format(src,dst,src_sym))
letters =  alphabet

for i in range(0, len(letters)):
    format_arc(src=0, dst=0, src_sym="<epsilon>", dst_sym=letters[i], w=1,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[i], w=0,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym="<epsilon>", w=1,f=f)
    for j in range(0, len(letters)):
        if(j!=i):
            format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[j], w=1,f=f)   
f.write('0\n')
f.close()
    

#TODO: change the edit distance

In [154]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms  lev.fst lev.bin.fst
!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait lev.bin.fst | dot -Tjpg >lev.jpg

An acceptor is a finite automaton where each transition has a label and possibly a weight. In this library, an acceptor is represented as a transducer with the input and output label of each transition being equal. 

In [155]:
#Step 6

f = open("acceptor.fst","w+")
s = 0
final_states = []
word_corpus_cut = word_corpus
for word in word_corpus_cut:
    format_arc(0,s+1,word[0],_,_,f,acceptor=True)
    s += 1
    for letter in word[1:]:
        format_arc(s,s+1,letter,_, _,f,acceptor=True)
        s += 1
    final_states.append(s)
    
for state in final_states:
    f.write(f'{state}\n')
    
f.close()    

In [156]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms --acceptor acceptor.fst acc.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait rosebud.bin1.fst | dot -Tjpg >acc.jpg

In [157]:
#Step 6b

!fstrmepsilon acc.bin.fst eps.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait eps.fst | dot -Tjpg >dfa_rmepsilon.jpg

!fstdeterminize eps.fst det.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait det.fst | dot -Tjpg >dfa_determinize.jpg

!fstminimize det.fst mini.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait mini.fst | dot -Tjpg >dfa_minimize.jpg


In [158]:
#Step 7
#mini.fst - lev.fst

!fstarcsort --sort_type=olabel lev.bin.fst lev_sorted.bin.fst
!fstarcsort --sort_type=ilabel mini.fst mini.fst

In [159]:
!fstcompose  lev_sorted.bin.fst mini.fst composed.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait composed.bin.fst | dot -Tjpg >com.jpg

In [160]:

f = open("input.fst","w+")
s = 0
final_states = []
word_corpus_cut = ['cit']
for word in word_corpus_cut:
    format_arc(0,s+1,word[0],word[0],0,f,acceptor=True)
    s += 1
    for letter in word[1:]:
        format_arc(s,s+1,letter, letter, 0,f,acceptor=True)
        s += 1
    final_states.append(s)
    
for state in final_states:
    f.write(f'{state}\n')
f.close()    

In [161]:
!fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor input.fst I.bin.fst

In [162]:
!fstarcsort --sort_type=ilabel composed.bin.fst s_composed.bin.fst 
!fstarcsort --sort_type=olabel I.bin.fst s_I.bin.fst 

In [163]:
!fstcompose s_I.bin.fst s_composed.bin.fst |fstshortestpath --nshortest=1 \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
| cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n'


pit

In [164]:
!fstcompose s_I.bin.fst s_composed.bin.fst |fstshortestpath --nshortest=1   \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms

0	1	c	p	1
1	2	i	i
2	3	t	t
3


In [165]:
#!wget https://raw.githubusercontent.com/georgepar/python-lab/master/spell_checker_test_set -P ./data/

In [166]:
# Step 8:
import random

file = open('./data/spell_checker_test_set',"r")
y_test = []
X_test = []
for line in file:
    [y,X] = line.split(':')
    X = X.split()
    for word in X:
        X_test.append(word)
        y_test.append(y)

print(y_test[0:4])
print(X_test[0:4])
print(len(y_test))
print(len(X_test))
idxs = random.sample(range(0, len(y_test)), 20)
print(idxs)


['contented', 'contented', 'contented', 'contented']
['contenpted', 'contende', 'contended', 'contentid']
270
270
[161, 251, 84, 82, 47, 119, 260, 257, 52, 232, 108, 222, 45, 100, 193, 143, 261, 177, 42, 121]


In [167]:
def predict(Y,X):
    for y,x in zip(Y,X):
        
        f = open("input.fst","w")
        s = 0
        word = x
        final_states = []
        format_arc(0,s+1,word[0],word[0],0,f,acceptor=True)
        s += 1
        for letter in word[1:]:
            format_arc(s,s+1,letter, letter, 0,f,acceptor=True)
            s += 1
        final_states.append(s)

        for state in final_states:
            f.write(f'{state}\n')
        f.close()    
        !fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor input.fst I.bin.fst
        !fstarcsort --sort_type=ilabel composed.bin.fst s_composed.bin.fst 
        !fstarcsort --sort_type=olabel I.bin.fst s_I.bin.fst 
        prediction = !fstcompose s_I.bin.fst s_composed.bin.fst |fstshortestpath --nshortest=1 \
        | fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
        | cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n' 
        print("Wrong = ",x,"-- Correct = ",y,"-- Prediction = ",prediction[0])
        correct_pred=0
        if y == prediction[0]:
            correct_pred+=1
    print(f"Accuracy:{correct_pred/len(Y)}%")

In [168]:
X_rand = [X_test[i] for i in idxs]
Y_rand = [y_test[i] for i in idxs]
predict(Y_rand,X_rand)

Wrong =  perpul -- Correct =  purple -- Prediction =  peril
Wrong =  oppossitte -- Correct =  opposite -- Prediction =  opposite
Wrong =  southen -- Correct =  southern -- Prediction =  southern
Wrong =  undersand -- Correct =  understand -- Prediction =  understand
Wrong =  avaible -- Correct =  available -- Prediction =  audible
Wrong =  latiest -- Correct =  latest -- Prediction =  latest
Wrong =  adres -- Correct =  address -- Prediction =  are
Wrong =  curtians -- Correct =  curtains -- Prediction =  martians
Wrong =  necassary -- Correct =  necessary -- Prediction =  necessary
Wrong =  extreamly -- Correct =  extremely -- Prediction =  extremely
Wrong =  beetween -- Correct =  between -- Prediction =  between
Wrong =  carrer -- Correct =  career -- Prediction =  carrier
Wrong =  buiscits -- Correct =  biscuits -- Prediction =  biscuits
Wrong =  desicate -- Correct =  desiccate -- Prediction =  delicate
Wrong =  accesing -- Correct =  accessing -- Prediction =  accepting
Wrong =  